# GDMA - Assignment 3

Author: Julian Schelb (1069967)

In [11]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Connection to the database instance

In [12]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "subatomic-shrank-Respond"))
session = driver.session()

### Question 1

Import the graph from the attached karate.csv file. Then, using functions from the Graph Data Science Library (GDS), write Cypher queries to compute the Betweenness, Closeness, and Eigenvector centrality of each
node. You can find the full description of the functions at:

https://neo4j.com/docs/graph-data-science/current/algorithms/centrality/

**Import Data:**

In [9]:
query = """
LOAD CSV FROM 'file:///karate.csv' AS row FIELDTERMINATOR ';'
WITH row[0] as sourceId, row[1] as targetId
MERGE (s:Node {id: sourceId})
MERGE (t:Node {id: targetId})
MERGE (s)-[:RELATED_TO]->(t)   
RETURN s, t
"""

with driver.session() as session:
    result = session.run(query)

**Create In-Memory Graph Projection:**

In [15]:
query = """
CALL gds.graph.project('tmpGraph', 'Node', 'RELATED_TO')
"""

with driver.session() as session:
    result = session.run(query)

**Betweenness Centrality:**

In [35]:
query = """
CALL gds.betweenness.stream('tmpGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score
MATCH (n:Node) 
WHERE  n.id = nodeId
SET n.betweenness = score
RETURN n.id, score
ORDER BY score DESC
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head()

,n.id,score
0,3,8.833333
1,32,5.083333
2,9,2.250000
3,29,2.166667
4,4,2.000000


**Eigenvector Centrality:**

In [41]:
query = """
CALL gds.eigenvector.stream('tmpGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score
MATCH (n:Node) 
WHERE  n.id = nodeId
SET n.eigenvector = score
RETURN n.id, score
ORDER BY score DESC
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head()

,n.id,score
0,34,0.993455
1,33,0.113378
2,14,0.006594
3,8,0.006594
4,13,0.006138


**Closeness Centrality:**

In [40]:
query = """
CALL gds.beta.closeness.stream('tmpGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score
MATCH (n:Node) 
WHERE  n.id = nodeId
SET n.closeness = score
RETURN n.id, score
ORDER BY score DESC
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(20)

,n.id,score
0,2,1.000000
1,3,1.000000
2,5,1.000000
3,6,1.000000
4,4,1.000000
5,7,1.000000
6,26,1.000000
7,30,1.000000
8,14,1.000000
9,20,1.000000


**Result:**

In [44]:
query = """
MATCH (a)
RETURN a.id, a.betweenness, a.eigenvector, a.closeness
"""        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,a.id,a.betweenness,a.eigenvector,a.closeness
0,1,0.000000,8.660452e-09,0.000000
1,2,0.500000,6.456418e-07,1.000000
2,3,8.833333,2.195913e-05,1.000000
3,5,0.000000,6.456418e-07,1.000000
4,6,0.500000,6.456418e-07,1.000000
5,4,2.000000,4.568230e-04,1.000000
6,7,1.500000,4.327262e-05,1.000000
7,24,0.000000,8.660452e-09,0.000000
8,25,0.000000,8.660452e-09,0.000000
9,27,0.000000,8.660452e-09,0.000000


***

Finally, write a Cypher query that removes the node from the graph with
the highest centrality only if the graph is weakly connected. The check
whether the graph is weakly connected must be done with Cypher as well.
Also, write a python function that executes the Cypher query repeatedly
until the graph becomes disconnected. Ideally, the python code must
execute only a single Cypher query per iteration.

In [50]:
query = """
MATCH (n)
WITH n as node, n.eigenvector as centrality
RETURN node, centrality
ORDER BY centrality DESC
"""        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,node,centrality
0,"(closeness, betweenness, id, betw_centr, eigen...",9.934551e-01
1,"(closeness, betweenness, id, betw_centr, eigen...",1.133783e-01
2,"(closeness, betweenness, id, betw_centr, eigen...",6.593799e-03
3,"(closeness, betweenness, id, betw_centr, eigen...",6.593799e-03
4,"(closeness, betweenness, id, betw_centr, eigen...",6.137621e-03
5,"(closeness, betweenness, id, betw_centr, eigen...",5.746022e-03
6,"(closeness, betweenness, id, betw_centr, eigen...",5.744748e-03
7,"(closeness, betweenness, id, betw_centr, eigen...",8.697364e-04
8,"(closeness, betweenness, id, betw_centr, eigen...",4.568230e-04
9,"(closeness, betweenness, id, betw_centr, eigen...",4.361465e-04
